In [1]:
from o2o import BivariateHawkesProcessSimulator, ParameterEstimator, O2OAnalyzer
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("combined_data.csv", low_memory=False, na_values=['', ' '])
cols_to_drop = ["OFFENSE_CODE_GROUP", "UCR_PART"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")
df = df.dropna(subset=["Location", "DISTRICT"])
print(f"Cleaned dataset loaded into df, shape = {df.shape}")

Cleaned dataset loaded into df, shape = (410002, 15)


In [3]:
# Steven's function to put data in correct format
def parse_dates_multi_utc(col: pd.Series) -> pd.Series:
    s = col.astype(str).str.strip()
    s = (s.str.replace(r"[…]|\.{3}", "", regex=True)
           .str.replace(r"\s+", " ", regex=True))

    # parse with utc=True every time so results are homogeneous tz-aware
    parsed = pd.to_datetime(s, format="%Y-%m-%d", errors="coerce", utc=True)

    for fmt in [
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d %I:%M %p",
        "%m/%d/%Y",
        "%m/%d/%Y %H:%M",
        "%m/%d/%Y %H:%M:%S",
    ]:
        mask = parsed.isna()
        if mask.any():
            parsed.loc[mask] = pd.to_datetime(s[mask], format=fmt, errors="coerce", utc=True)

    # final fallback (dateutil)
    mask = parsed.isna()
    if mask.any():
        parsed.loc[mask] = pd.to_datetime(s[mask], errors="coerce", utc=True)

    return parsed  # dtype should be datetime64[ns, UTC]

In [4]:
category_column = df['OFFENSE_DESCRIPTION']
unique_offenses = category_column.dropna().unique()

M = 1 
df0 = df.copy()

cutoff1 = pd.Timestamp('2025-04-01')
cutoff2 = pd.Timestamp('2025-10-01')  

offense_list = []
spillover_list = []
decay_list = []
percentages_list = []
base_list = []

#Iterating through each unique crime type (note the second district says D4, but was set to equal C6 (correct district)

for offense in unique_offenses:
    print(offense)
    df_new = df0.loc[df0["OFFENSE_DESCRIPTION"] == offense].copy()
    if df_new.empty:
        continue

    df_new['OCCURRED_ON_DATE'] = parse_dates_multi_utc(df_new['OCCURRED_ON_DATE'])
    df_new['OCCURRED_ON_DATE'] = df_new['OCCURRED_ON_DATE'].dt.tz_convert('UTC').dt.tz_localize(None)

    df_timewindow = df_new[
        (df_new['OCCURRED_ON_DATE'] >= cutoff1) &
        (df_new['OCCURRED_ON_DATE'] <= cutoff2)
    ].copy()
    if df_timewindow.empty:
        continue

    B2 = df_timewindow[df_timewindow['DISTRICT'] == 'B2'].copy()
    D4 = df_timewindow[df_timewindow['DISTRICT'] == 'C6'].copy()
    if B2.empty or D4.empty:
        continue

    t0_B2 = B2['OCCURRED_ON_DATE'].min()
    t0_D4 = D4['OCCURRED_ON_DATE'].min()
    if pd.isna(t0_B2) or pd.isna(t0_D4):
        continue

    span_days = max(
        (B2['OCCURRED_ON_DATE'].max() - B2['OCCURRED_ON_DATE'].min()).days,
        (D4['OCCURRED_ON_DATE'].max() - D4['OCCURRED_ON_DATE'].min()).days
    )
    T = max(span_days, 1)


    B2['t'] = (B2['OCCURRED_ON_DATE'] - t0_B2).dt.total_seconds() / 86400.0
    D4['t'] = (D4['OCCURRED_ON_DATE'] - t0_D4).dt.total_seconds() / 86400.0

    data_B2 = [sorted(B2['t'].dropna().tolist())]
    data_D4 = [sorted(D4['t'].dropna().tolist())]

    estimator = ParameterEstimator(data_B2, data_D4, M, T)
    fit = estimator.fit_model()
    spillover, decay = estimator.spillover_and_decays_values_and_CI()
    percentages = estimator.spillover_percentage()
    base_estimation = estimator.base_and_CI()

    offense_list.append(offense)
    spillover_list.append(spillover)
    decay_list.append(decay)
    percentages_list.append(percentages)
    base_list.append(base_estimation)


FRAUD - FALSE PRETENSE / SCHEME


12:34:17 - cmdstanpy - INFO - CmdStan start processing


chain 1:   0%|          | 0/1500 [00:00<?, ?it/s, (Warmup)]

12:34:22 - cmdstanpy - INFO - CmdStan done processing.



FRAUD - IMPERSONATION
SICK/INJURED/MEDICAL - PERSON


...


OPERATING UNDER THE INFLUENCE (OUI) DRUGS
RECOVERED - MV RECOVERED IN BOSTON (STOLEN IN BOSTON) MUST BE SUPPLEMENTAL
Justifiable Homicide
TRUANCY / RUNAWAY
POSSESSION OF BURGLARIOUS TOOLS
EXPLOSIVES - TURNED IN OR FOUND
OBSCENE PHONE CALLS
PROSTITUTION - ASSISTING OR PROMOTING
Evidence Tracker Incidents
LARCENY THEFT FROM COIN-OP MACHINE
MANSLAUGHTER - NEGLIGENCE
FIREARM/WEAPON - LOST


14:10:14 - cmdstanpy - INFO - CmdStan start processing


chain 1:   0%|          | 0/1500 [00:00<?, ?it/s, (Warmup)]

14:10:15 - cmdstanpy - INFO - CmdStan done processing.



MANSLAUGHTER - VEHICLE - NEGLIGENCE
FIREARM/WEAPON - ACCIDENTAL INJURY / DEATH
MURDER, NON-NEGLIGENT MANSLAUGHTER
PRISONER ESCAPE / ESCAPE & RECAPTURE


In [5]:
print(len(spillover_list))
print(len(offense_list))

82
82


In [6]:
print(spillover_list)

[                         Effect  Value          95% CI
0    Online to Online Spillover  0.070  [0.003, 0.228]
1  Offline to Offline Spillover  0.095  [0.011, 0.234]
2   Online to Offline Spillover  0.107  [0.015, 0.256]
3   Offline to Online Spillover  0.098  [0.005, 0.324],                          Effect  Value          95% CI
0    Online to Online Spillover  0.118  [0.009, 0.322]
1  Offline to Offline Spillover  0.119  [0.013, 0.260]
2   Online to Offline Spillover  0.112  [0.007, 0.336]
3   Offline to Online Spillover  0.065  [0.005, 0.181],                          Effect  Value          95% CI
0    Online to Online Spillover  0.146  [0.005, 0.464]
1  Offline to Offline Spillover  0.277  [0.057, 0.621]
2   Online to Offline Spillover  0.138  [0.009, 0.421]
3   Offline to Online Spillover  0.144  [0.007, 0.422],                          Effect  Value          95% CI
0    Online to Online Spillover  0.213  [0.028, 0.518]
1  Offline to Offline Spillover  0.194  [0.012, 0.478]
2   On

In [7]:
roxbury_to_SouthBos= []
southbos_to_roxbury = []
CI_RS = []
CI_SR =[]
#getting the values and CI for the spillover 
for df in spillover_list:
    value1 = df.loc[2, 'Value'] 
    value2 = df.loc[3, 'Value'] 
    CI_value1 = df.loc[2, "95% CI"] 
    CI_value2 = df.loc[3, "95% CI"] 
    roxbury_to_Southbos.append(value1)
    southbos_to_roxbury.append(value2)
    CI_RS.append(CI_value1)
    CI_SR.append(CI_value2)

#get the CI in correct format
CI_cleaned1 =[]

for row in CI_RS:
    numbers = []
    for item in row:
        try:
            num = float(item)
            numbers.append(num)
        except:
            pass

    CI_cleaned1.append(numbers)

CI_cleaned2 =[]

for row in CI_SR:
    numbers = []
    for item in row:
        try:
            num = float(item)
            numbers.append(num)
        except:
            pass

    CI_cleaned2.append(numbers)

    

In [8]:
# Making dataframe of values (unsorted/sorted)
roxbury_to_Southbos = [float(s) for s in roxbury_to_Southbos]
southbos_to_roxbury = [float(s) for s in southbos_to_roxbury]

data = {'Offense': offense_list, 'Spillover Roxbury to South Boston': roxbury_to_Southbos, 'Spillover South Boston to Roxbury':southbos_to_roxbury}

data2 = {'Offense': offense_list, 'Spillover Roxbury to South Boston': roxbury_to_Southbos}
data3 = {'Offense': offense_list,'Spillover South Boston to Roxbury':southbos_to_roxbury}


spilloverdf = pd.DataFrame(data)
spilloverdf2 = pd.DataFrame(data2)
spilloverdf3 = pd.DataFrame(data3)


spilloverdf['CI 95% Roxbury to South Boston'] = CI_cleaned1
spilloverdf['CI 95% South Boston to Roxbury'] = CI_cleaned2
spilloverdf2['CI 95% Roxbury to South Boston'] = CI_cleaned1
spilloverdf3['CI 95% South Boston to Roxbury'] = CI_cleaned2


print(spilloverdf)
sorted_spilloverdf2 = spilloverdf2.sort_values(by='Spillover Roxbury to South Boston', ascending=False)
sorted_spilloverdf3 = spilloverdf3.sort_values(by='Spillover South Boston to Roxbury', ascending=False)

spilloverdf.to_csv('/O2O/O2O/crimespilloverSouthBoston_notsorted.csv')
sorted_spilloverdf2.to_csv('/O2O/O2O/crimespilloverSouthBoston_sortedroxbury.csv')
sorted_spilloverdf3.to_csv('/O2O/O2O/crimespilloverSouthBoston_sortedSouthBoston.csv')





                                    Offense  \
0           FRAUD - FALSE PRETENSE / SCHEME   
1             SICK/INJURED/MEDICAL - PERSON   
2                  FORGERY / COUNTERFEITING   
3               LARCENY THEFT FROM BUILDING   
4                        INVESTIGATE PERSON   
..                                      ...   
77  DRUGS - POSSESSION OF DRUG PARAPHANALIA   
78                                    ARSON   
79                              BOMB THREAT   
80                PROSTITUTION - SOLICITING   
81                    FIREARM/WEAPON - LOST   

    Spillover Roxbury to South Boston  Spillover South Boston to Roxbury  \
0                               0.107                              0.098   
1                               0.112                              0.065   
2                               0.138                              0.144   
3                               0.037                              0.037   
4                               0.054                  

In [33]:
import os
current_directory = os.getcwd()
print(f"Current Working Directory: {current_directory}")

Current Working Directory: /Users/meganhogan/O2O
